In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchmetrics import AUROC
import pandas as pd

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, RobertaModel, RobertaTokenizer, get_linear_schedule_with_warmup
from transformers import AutoConfig, AutoTokenizer

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42
rcParams['figure.figsize'] = 12, 8
pl.seed_everything(RANDOM_SEED)

In [ ]:
device = torch.device('cpu')

### Audio embeddings 

In [ ]:
train_wav_PATH = "C:/Data/Sentiment Analysis/MELD/Processed/wav2vec/embeddings_v2/train_wav.pt"
dev_wav_PATH = "C:/Data/Sentiment Analysis/MELD/Processed/wav2vec/embeddings_v2/dev_wav.pt"
test_wav_PATH = "C:/Data/Sentiment Analysis/MELD/Processed/wav2vec/embeddings_v2/test_wav.pt"

In [ ]:
train_wav_embeddings = torch.load(train_wav_PATH, map_location=device)
dev_wav_embeddings = torch.load(dev_wav_PATH, map_location=device)
test_wav_embeddings = torch.load(test_wav_PATH, map_location=device)

In [ ]:
train_wav_embeddings["embeddings"][0]

## Train 

In [ ]:
# List of data
train_dataset = []

# i is the index of arrays
for i in range(len(train_wav_embeddings["fileID"])):
    train_dataset.append({
        "fileID": train_wav_embeddings["fileID"][i], 
        "label": train_wav_embeddings["labels"][i], 
        "wav_embeddings": train_wav_embeddings["embeddings"][i][0],
    })

In [ ]:
len(train_dataset)

## Dev

In [ ]:
# List of data
dev_dataset = []

# i is the index of arrays
for i in range(len(dev_wav_embeddings["fileID"])):
    dev_dataset.append({
        "fileID": dev_wav_embeddings["fileID"][i], 
        "label": dev_wav_embeddings["labels"][i], 
        "wav_embeddings": dev_wav_embeddings["embeddings"][i][0],
    })

In [ ]:
len(dev_dataset)

## Test 

In [ ]:
# List of data
test_dataset = []

# i is the index of arrays
for i in range(len(test_wav_embeddings["fileID"])):
    test_dataset.append({
        "fileID": test_wav_embeddings["fileID"][i], 
        "label": test_wav_embeddings["labels"][i], 
        "wav_embeddings": test_wav_embeddings["embeddings"][i][0],
    })

In [ ]:
len(test_dataset)

## Encapsulating all data 

In [ ]:
class DataModule(pl.LightningDataModule):
    
    def __init__(self, train_dataset, test_dataset, dev_dataset, batch_size=16):
        super().__init__()
        self.batch_size = batch_size
        self.train = train_dataset
        self.dev = dev_dataset
        self.test = test_dataset

    def train_dataloader(self):
        return DataLoader(
          self.train,
          batch_size=self.batch_size,
          shuffle=False,
          num_workers=0
        )

    def val_dataloader(self):
        return DataLoader(
          self.dev,
          batch_size=self.batch_size,
          num_workers=0
        )

    def test_dataloader(self):
        return DataLoader(
          self.test,
          batch_size=self.batch_size,
          num_workers=0
        )

#### Instance for class DataModule 

In [ ]:
N_EPOCHS = 100
BATCH_SIZE = 16

data_module = DataModule(train_dataset, test_dataset, dev_dataset,batch_size=BATCH_SIZE)

### Modelling Prep 

In [ ]:
#train_dataset[0]["wav_embeddings"].size()
test_dataset[0]["wav_embeddings"].shape

## Modelling 

In [ ]:
class Tagger(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        
        self.linear_1 = nn.Linear(768, 668)
        self.linear_2 = nn.Linear(668, 568)
        self.linear_3 = nn.Linear(568, 468)
        self.linear_4 = nn.Linear(468, 368)
        self.linear_5 = nn.Linear(368, 256)
        self.classifier = nn.Linear(256, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.CrossEntropyLoss()
        

    def forward(self, wav_embeddings, labels=None):

        output = wav_embeddings
        #print(output)
        #print(output.shape)
        output = self.linear_1(output)
        output = self.linear_2(output)
        output = self.linear_3(output)
        output = self.linear_4(output)
        output = self.linear_5(output)
        output = self.classifier(output)
        output = torch.softmax(output, dim=1)
        loss = 0
        
        #Print to debug
        #print(output.shape)
        #print(labels)
        
        if labels is not None:
            labels = labels.flatten() ##
            loss = self.criterion(output, labels)
        
        return loss, output

    def training_step(self, batch, batch_idx):

        audio_embeddings = batch["wav_embeddings"]
        labels = batch["label"]
        loss, outputs = self(audio_embeddings, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        
        audio_embeddings = batch["wav_embeddings"]
        labels = batch["label"]
        loss, outputs = self(audio_embeddings, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):

        audio_embeddings = batch["wav_embeddings"]
        labels = batch["label"]
        loss, outputs = self(audio_embeddings, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_epoch_end(self, outputs):

        labels = []
        predictions = []

        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        pred = torch.argmax(predictions, dim=1)
        
        train_acc = accuracy(pred, labels, num_classes=3)
        #print("Label:", labels)
        #print("Prediction:", pred)
        print("Training Accuracy:", train_acc)
        
        label = labels.flatten()
        auroc = AUROC(num_classes=3)
        auroc = auroc(predictions, label)
        print("AUROC:", auroc)

    def configure_optimizers(self): #configuring the optimizers

        optimizer = AdamW(self.parameters(), lr=2e-5)

        scheduler = get_linear_schedule_with_warmup(
          optimizer,
          num_warmup_steps=self.n_warmup_steps,
          num_training_steps=self.n_training_steps

        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )

        )

In [ ]:
steps_per_epoch=len(train_dataset) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [ ]:
#1/5 of the training steps as warm-up
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

#### Instance for modelling class Tagger 

In [ ]:
model = Tagger(
  n_classes=3,
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps
)

## Training 

In [ ]:
#Checkpointing that saves the best model (based on validation loss)
checkpoint_callback = ModelCheckpoint(
    dirpath="C:/Users/id301281/NLP/NLU/MELD/Fusion",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
    )

In [ ]:
#And early stopping triggers when the loss hasn’t improved for the last 30 epochs
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=30)

In [ ]:
#starting training
trainer = pl.Trainer(
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

In [ ]:
torch.cuda.is_available()

In [ ]:
trainer.fit(model, data_module)

### Evaluation 

In [ ]:
trainer.test()

In [ ]:
#loading best trained model
trained_model = Tagger.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=3
)

trained_model.eval()
trained_model.freeze()

In [ ]:
preds = [] 
labels = []

for i in test_dataset:
    
    wav_embeddings = i["wav_embeddings"].unsqueeze(0)
    actual = i['label']    
    pred1 = trained_model(wav_embeddings)
    pred2 = torch.argmax(pred1[1], dim=1).squeeze().tolist()
    
    preds.append(pred2)
    labels.append(actual)

In [ ]:
encode_map = {
    
    "negative": "0",
    "neutral": "1",
    "positive": "2"
}

In [ ]:
print(classification_report(labels, preds, target_names=encode_map))

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(labels, preds)
df_cm = pd.DataFrame(cm, index=encode_map, columns=encode_map)
show_confusion_matrix(df_cm)

### Storing test predictions: 

In [ ]:
#loading best trained model
trained_model = Tagger.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=3
)

trained_model.eval()
trained_model.freeze()

In [ ]:
test_prediction = trained_model(test_dataset[0]["wav_embeddings"].unsqueeze(0))

In [ ]:
preed = test_prediction[1].flatten().numpy()
for i, predd in zip(encode_map, preed):
    print(f"{i}: {predd}")

In [ ]:
test_dataset[0]['wav_embeddings'].unsqueeze(0).size()

In [ ]:
test_predictions = { "fileID_w2vb": [], "Negative_w2vb" : [], "Neutral_w2vb" : [], "Positive_w2vb" : [], "predicted_w2vb": [], "actual_w2vb": []}

for i in test_dataset:
    
    wav_embeddings = i["wav_embeddings"].unsqueeze(0)
    actual = i['label']
    fileID = i['fileID']
    
    pred1 = trained_model(wav_embeddings)
    pred2 = torch.argmax(pred1[1], dim=1).squeeze().tolist()
    pred3 = pred1[1].flatten().numpy()
    
    test_predictions["predicted_w2vb"].append(pred2)
    test_predictions["Negative_w2vb"].append(pred3[0])
    test_predictions["Neutral_w2vb"].append(pred3[1])
    test_predictions["Positive_w2vb"].append(pred3[2])
    test_predictions["actual_w2vb"].append(actual)
    test_predictions["fileID_w2vb"].append(fileID)

In [ ]:
df = pd.DataFrame(data=test_predictions)
df

In [ ]:
#saving to csv
df.to_csv(r"C:\Data\Sentiment Analysis\MELD\ensemble preds\wav2vec\v2\test_preds_w2vb.csv")